Practical Work in AI - Concept Models
Tragler Thomas
====================


In [1]:
import pandas as pd
import os
import torch
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import wandb
from datetime import datetime
import torch.optim as optim
from sklearn.model_selection import KFold

import derm7pt_data
from derm7pt_data import Derm7pt_data

from importlib import reload


reload(derm7pt_data)

path = os.path.normpath('Data\\Derm7pt')

derm7pt = Derm7pt_data(path)
metadata = derm7pt.metadata
print(metadata.shape)

(1011, 20)


In [2]:
#Data loading
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)
my_dataset = metadata

#split data into training, validation and test sets
train_size = int(0.8 * len(my_dataset))
val_size = int(0.15 * len(my_dataset))
test_size = len(my_dataset) - train_size - val_size
#ToDo stratified sampling
#overall weird spliting ToDo
#train_split, val_split, test_split = random_split(my_dataset, [train_size, val_size, test_size])

#train_data, train_concepts, train_label = derm7pt.data_split_by_index(train_split.indices)
#val_data, val_concepts, val_label = derm7pt.data_split_by_index(val_split.indices)
#test_data, test_concepts, test_label = derm7pt.data_split_by_index(test_split.indices)

#Dataload 
#train_loader = DataLoader(derm7pt, batch_size=batch_size, shuffle=True)


device: cuda


In [3]:
my_dataset.columns

Index(['case_num', 'diagnosis', 'seven_point_score', 'pigment_network',
       'streaks', 'pigmentation', 'regression_structures', 'dots_and_globules',
       'blue_whitish_veil', 'vascular_structures',
       'level_of_diagnostic_difficulty', 'elevation', 'location', 'sex',
       'clinic', 'derm', 'nums', 'is_cancer', 'abbrevs', 'info'],
      dtype='object')

In [4]:
#Torch CNN model with 3 Conv layers and 3 fully connected layers
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #conv Layers
        in_channels, out_channels = (3, 16)
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1)
        in_channels, out_channels = (out_channels, 2*out_channels)
        self.conv2 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1)
        # in_channels, out_channels = (out_channels, 2*out_channels)
        # self.conv3 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1)
        
        #Max pooling layers
        self.pool = nn.MaxPool2d(kernel_size=4, stride=4, padding=0)
        
        self.channels_before_linear = out_channels
        # Fully connected layers
        self.fc1 = nn.Linear(self.channels_before_linear * 192 * 128, 128)  # Adjust the input size based on your image dimensions
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 1)  # Output layer with one neuron for binary classification
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Convolutional layers with activation and pooling
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        # x = self.relu(self.conv3(x))
        
        #Pooling layer
        x = self.pool(x)

        # Flatten the output for fully connected layers
        x = x.view(-1, self.channels_before_linear * 192 * 128)  # Corrected input size based on spatial dimensions

        # Fully connected layers with activation
        x = self.relu(self.fc1(x))
        # x = self.relu(self.fc2(x))
        x = self.fc3(x)
        x = x

        return x
    

In [5]:
def majority_class_baseline(true_labels):
    # Find the most frequent class in the training set
    elems, counts = true_labels.unique(return_counts=True)
    majority_count = counts[counts.argmax()]
    majority_class = elems[counts.argmax()]
    #predictions = torch.full_like(true_labels, majority_class)
    accuracy = majority_count / len(true_labels)
    return majority_class, accuracy

In [7]:
#Training the model

# hyperparameters
n_epochs = 10
learning_rate = 0.001
n_folds = 8
batch_size = 64

criterion = nn.BCEWithLogitsLoss()

wandb.init(
    # set the wandb project where this run will be logged
    project= "PracticalWork",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": learning_rate,
    "architecture": "SimpleCNN",
    "dataset": "derm7pt",
    "labels": derm7pt.model_columns["label"],
    "epochs": n_epochs,
    "batch_size": batch_size,
    "n_folds": n_folds,
    "device": device
    },
    name="run"+str(datetime.now())
)


#Todo
simple_val_baseline = 0.5

# Training loop
kf = KFold(n_splits=n_folds, shuffle=True)
for fold, (train_idx, val_idx) in enumerate(kf.split(derm7pt.metadata)):
    #start of time measurement
    #Todo Hackfix to get the majority class of the validation set
    print(datetime.now())
    if True:
        val_loader = DataLoader(
            dataset=derm7pt,
            batch_size=999999,
            sampler=torch.utils.data.SubsetRandomSampler(val_idx),
        )
        for i, batch in enumerate(val_loader, 0):
            inputs, labels = batch
            _, simple_val_baseline = majority_class_baseline(labels)
    print("finished ", datetime.now())
    train_loader = DataLoader(
        dataset=derm7pt,
        batch_size=batch_size,
        sampler=torch.utils.data.SubsetRandomSampler(train_idx),
    )
    val_loader = DataLoader(
        dataset=derm7pt,
        batch_size=batch_size,
        sampler=torch.utils.data.SubsetRandomSampler(val_idx),
    )
    
    # Instantiate the model
    model = Net()
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    
    for epoch in range(n_epochs):
        running_loss = 0.0
        i = 0
        for i, batch in enumerate(train_loader, 0):
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)
    
            # Zero the parameter gradients
            optimizer.zero_grad()
    
            # Forward pass, backward pass, and optimization
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
            # Print statistics
            running_loss += loss.item()
            # if i % 10 == 9:    # Print every 10 mini-batches
            #     print('[%d, %5d] loss: %.3f' %
            #           (epoch + 1, i + 1, running_loss / 10))
            #     running_loss = 0.0
            
        running_loss /= (i+1)        
        
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for i, batch in enumerate(val_loader, 0):
                images, labels = batch
                images, labels = images.to(device), labels.to(device)
                output = model(images)
                output = torch.sigmoid(output) > 0.5
                total += labels.size(0)
                correct += (output == labels).sum().item()
                #print("outputs: ", output, total, correct)
                break
                
        
        val_accuracy = correct/total
        wandb.log({"loss": running_loss, "validation_accuracy": val_accuracy})
        print('[%d, %5d] loss: %.4f, val_accuracy: %.4f, simple_baseline: %.4f' % (epoch + 1, i + 1, running_loss, val_accuracy, simple_val_baseline))   
        
        
        #ToDo early stopping
        if running_loss < 0.1:
            print("Early stopping!")
            break
    
    #ToDo only one fold for now
    break
wandb.finish()
print('Finished Training')

loss,▁
validation_accuracy,▁
loss,2.7581
validation_accuracy,0.375


2024-03-28 21:34:59.394725
finished  2024-03-28 21:35:00.646575
(tensor(0., device='cuda:0'), tensor(0.6406, device='cuda:0'))
(tensor(0., device='cuda:0'), tensor(0.7344, device='cuda:0'))
(tensor(0., device='cuda:0'), tensor(0.7500, device='cuda:0'))
(tensor(0., device='cuda:0'), tensor(0.6719, device='cuda:0'))
(tensor(0., device='cuda:0'), tensor(0.6562, device='cuda:0'))
(tensor(0., device='cuda:0'), tensor(0.7344, device='cuda:0'))
(tensor(0., device='cuda:0'), tensor(0.6250, device='cuda:0'))
(tensor(0., device='cuda:0'), tensor(0.6562, device='cuda:0'))
(tensor(0., device='cuda:0'), tensor(0.7500, device='cuda:0'))
(tensor(0., device='cuda:0'), tensor(0.6875, device='cuda:0'))
(tensor(0., device='cuda:0'), tensor(0.6875, device='cuda:0'))
(tensor(0., device='cuda:0'), tensor(0.6875, device='cuda:0'))
(tensor(0., device='cuda:0'), tensor(0.7969, device='cuda:0'))
(tensor(0., device='cuda:0'), tensor(0.8077, device='cuda:0'))
[1,     1] loss: 4.2932, val_accuracy: 0.6875, simple_

loss,█▁▁▁▁▁▁▁▁▁
validation_accuracy,▁▅▇▇▅▅▇█▃▄
loss,0.31192
validation_accuracy,0.76562


Finished Training
